# Example of spatiotemporal integrative modeling

Here, we walk through an example of spatiotemporal integrative modeling. For a toy model, we assume there are 2 proteins: A and B. We are interested in assembly of the trimer of these proteins.

For simplicity, we will assume that data suggests the assembly proceeds as a monomer->dimer->trimer, and that the data for these states was collected at set time points, namely 0 minutes, 5 minutes, and 10 minutes. Based on these assumptions, our model can be present in 6 possible states at 3 time points (2 states at 0 minues, 3 states at 5 minutes, and 2 states at 10 minutes), as summarized in the image below:

![title](image/toy_model.pdf)

For each of these combinations of proteins A and B, we would generate integrative models of each complex, following procedures such as those in modeling the Actin complex (https://integrativemodeling.org/tutorials/actin/). We will assume that this process was already done. The scores of each model were saved as *_scores.log and the stoicheometery for each model is described by *.config.

### Model without temporal scoring
Using just this data as an input, we can compute our spatiotemporal model with one line of code:

In [2]:
# import relevant modules
import sys
sys.path.insert(0, "../../")
import IMP.spatiotemporal as spatiotemporal
import os

# Always from in main_dir
main_dir=os.getcwd()
os.chdir(main_dir)

# Input variables. 
# dict is a the state dictionary, which returns the number of states at each time point, for each time point.
# Input is the directory where the data is stored. 
# Output is the directory to which the output will be written
dict={'0min':2,'5min':3,'10min':2}
input='data'
output='../output_notemp'

# create DAG
graph,graph_scores=spatiotemporal.create_DAG(dict,input_dir=input,output_dir=output)



Initialing graph...
Done.
Scoring directed acycling graph...
Done.
Writing output...
Done.


### Outputs and data analysis

#### Output files
Note that the folder output_notemp has 4 different files. Each of these files contains different ways of describing or visualizing the graph.

cdf.txt - stores the cummulative distribution function (cdf) over all trajectories, with each row corresponding to the next most likely trajectory being added to the total cdf.

labeled_pdf.txt - stores the probability distribution function over all trajectories. Each row has 2 columns. The first column describes each state along the trajectory, divided by '|'. The next column is the probability of that trajectory.

dag_heatmap/dag_heatmap.eps - visualization of the graph as a heatmap, created by graphviz. By default, each state is represented as a sphere with darker spheres corresponding to higher probability.

### Model with temporal scoring
Closer analysis of the output from the analysis above shows that not trajectories are purely associative, meaning proteins would have to dissociate from the assembling complex for that trajectory to be relevant. We can filter out these unwanted trajectories using the spatio_temporal_rule keyword:

In [3]:
# go to main_dir
os.chdir(main_dir)

# Input variables. 
dict={'0min':2,'5min':3,'10min':2}
input='data'
output='../output_temp'
# expected_subcomplexes is a list of all possible subcomplex strings in the model. Should match the configuration files
subcomplexes=['A1','A2','B1','B2']

graph,graph_scores=spatiotemporal.create_DAG(dict,input_dir=input,output_dir=output,spatio_temporal_rule=True,expected_subcomplexes=subcomplexes)

Initialing graph...
Done.
Scoring directed acycling graph...
Done.
Writing output...
Done.


### Adding stoichiometeric data
Sometimes, experiments such as fluorescence correlation spectroscopy (FCS) can provide time-dependent stoichiometeric data. In such cases, we can utilize this data to inform our spatiotemporal model. By adding the score_comp keyword, we can include a liklihood to our Bayesian scoring function that accounts for protein composition. Note that the precision of our model increased because of the additional data provided:

In [4]:
# go to main_dir
os.chdir(main_dir)

# Input variables. 
dict={'0min':2,'5min':3,'10min':2}
input='data'
output='../output_stoich'
subcomplexes=['A1','A2','B1','B2']
# exp_comp_map is a dictionary that describes protein stoicheometery. The key describes the protein, which should correspond to names within the expected_subcomplexes. For each of these proteins, a csv file should be provided with protein copy number data
exp_comp={'A':'exp_comp_A.csv','B':'exp_comp_B.csv'}


graph,graph_scores=spatiotemporal.create_DAG(dict,input_dir=input,output_dir=output,spatio_temporal_rule=True,expected_subcomplexes=subcomplexes,score_comp=True,exp_comp_map=exp_comp)

Initialing graph...
Done.
Calculation composition likelihood...
Done.
Scoring directed acycling graph...
Done.
Writing output...
Done.


### Precision of model is dependent on the precision of input data
The precision of the final model is dependent on the precision of the data used to construct the model. To demonstrate this point, we assume we can create a data set with more precise stoicheometeric data. Note that, now, the model converges onto a single trajectory. Here, we also demonstrate some other output options that may be useful:


In [5]:
# go to main_dir
os.chdir(main_dir)

# Input variables. 
dict={'0min':2,'5min':3,'10min':2}
input='data_precise'
output='../output_stoich_precise'
subcomplexes=['A1','A2','B1','B2']
exp_comp={'A':'exp_comp_A_precise.csv','B':'exp_comp_B_precise.csv'}


graph,graph_scores=spatiotemporal.create_DAG(dict,out_pdf=True,npaths=2,input_dir=input,output_dir=output,spatio_temporal_rule=True,expected_subcomplexes=subcomplexes,score_comp=True,exp_comp_map=exp_comp)

Initialing graph...
Done.
Calculation composition likelihood...
Done.
Scoring directed acycling graph...
Done.
Writing output...
Done.


### Other outputs
Note the 2 additional commands, out_pdf and npaths, which caused the creation of new analysis files 'pdf.txt' and 'path1.txt'/'path2.txt' respectively. These files describe:

pdf.txt - stores the probability distribution function (pdf) over all trajectories, with each row corresponding to the next most likely trajectory.

path1.txt / path2.txt - Each line corresponds to a single state visited in the most likely path (path1.txt) or 2nd most likely path (path2.txt). Note that 2 files were written because npaths=2.

## Summary of all options
The create_DAG function has a variety of options for model input and plotting. The tutorial above highlighted the key functions that are necessary to create spatio-temporal models, but other options are also availabe. Here, we highlight all options of the create_DAG function:

#### Inputs related to model input / calculation
state_dict - dictionary that defines the spatiotemporal model. They keys are strings that correspond to each time point in the stepwise temporal process.
Keys should be ordered according to the steps in the spatiotemporal process. The values are integers that correspond to the number of possible states at that timepoint.
Scores for each model are expected to be stored as state_timescorestr,
where state are integers 1->value of the dictionary, time is the key in the dictionary, and
scorestr is trailing characters, which are assumed to be constant for all states.

input_dir - string, directory where the data is stored. Empty string assumes current working directory.

scorestr - string, trailing characters at the end of the file with scores for each stage of the spatiotemporal model (default: '_scores.log').

output_dir - string, directory where the output will be written. Empty string assumes the same directory as the input_dir.

#### Inputs related to spatiotemporal scoring (all optional)
spatio_temporal_rule- Boolean. If true, enforces that all components earlier in the assembly process are present later in the process. (default: False)

subcomplexstr- string, trailing characters after the subcomplex file, which is a list of subcomplexes included in the given label/time (default: '.config')

expected_subcomplexes- list of all possible subcomplex strings in the model (default: []) Should be a list without duplicates of all components in the subcomplex files.

#### Inputs related to composition scores (all optional)
score_comp - Boolean to determine whether or not to score models based on the protein composition

exp_comp_map - dictionary for determining protein composition score. The keys are the proteins. The code checks if the name of these proteins are within the subcomplex_components for each node. As such, the naming scheme should be such that the keys of exp_comp_map are substrings of expected_subcomplexes the values of exp_comp_map should correspond to a csv file for each subcomplex with protein copy numbers. Each csv file should have 3 columns:
1) 'Time' - should correspond to the keys of state_dict, 2) 'mean' - mean copy number from experimental data, and 3) std - standard deviation from experimental data

#### Inputs related to model output (all optional)
out_cdf - Boolean to determine whether or not to write out the cumulative distribution function (cdf) for the graph (default: True)

cdf_fn - string, filename for the cdf (default: 'cdf.txt')

out_labeled_pdf - Boolean to determine whether to output the labeled pdf file, which includes both the pdf and the ordered states visited along each path (default: True).

labeled_pdf_fn - string, name of the file for the labeled pdf (default: 'labeled_pdf.txt')

out_pdf - Boolean to determine whether or not to write out the probability distribution function (pdf) for the graph (default: False)

pdf_fn - string, filename for the pdf (default: 'pdf.txt')

npaths - int, write out the states along the n most likely paths, based on the pdf (default: 0)

npath_fn - string, name of the file for each of the n most likely paths. 'n.txt' will be appended to the end of npath_fn (default: 'path')

#### Inputs related to directed acyclic graph (DAG) output (all optional)
draw_dag - Boolean to determine whether or not to write out a directed acyclic graph (dag) to a file (default: True)

dag_fn - string, filename for the dag image (default: 'dag_heatmap')

dag_heatmap - Boolean to determine whether or not to write the dag with a heatmap based on the probability of each state (default: True)

dag_colormap - string, colormap used by the dag to represent probability. Chooses from those available in matplotlib
(https://matplotlib.org/stable/users/explain/colors/colormaps.html) (default: "Purples").

dag_draw_label - Boolean to determine whether or not to draw state labels on the dag

dag_fontname - string, font used for the labels on the dag

dag_fontsize - string, font size used for the labels on the dag

dag_penscale - float, size of the pen used to draw arrows on the dag

dag_arrowsize - float, size of arrows connecting states on the dag

dag_height - string, height of each node on the dag

dag_width - string, width of each node on the dag